In [6]:
import os
import numpy as np

import sys
sys.path.append('../')
from utils import load_3dgs_data
from fusion_utils import preprocess_3dgs_attributes

def compute_3dgs_min_max(path_3dgs_root, use_features=('scale', 'opacity', 'rotation')):
    """
    모든 Scene에서 3DGS 속성을 수집하여 각 속성의 최대/최소값 계산.
    
    Args:
        path_3dgs_root (str): 3DGS 데이터가 저장된 루트 경로.
        use_features (tuple): 사용할 3DGS 속성 ('scale', 'opacity', 'rotation' 중 선택).
    
    Returns:
        dict: 각 속성의 최대/최소값을 포함하는 딕셔너리.
    """
    # 모든 Scene 폴더 목록
    scene_folders = [f for f in os.listdir(path_3dgs_root) 
                     if os.path.isdir(os.path.join(path_3dgs_root, f))]
    
    # 속성별 리스트 초기화
    all_scales = []
    all_opacities = []
    all_rotations = []

    # 각 Scene의 속성 수집
    for scene_folder in scene_folders:
        path_3dgs = os.path.join(path_3dgs_root, scene_folder, "point_cloud.ply")
        print(f"Processing {scene_folder}...")
        
        # 1. 3DGS 데이터 로드
        _, _, vertex_data_3dgs = load_3dgs_data(path_3dgs)

        # 2. 3DGS 속성 전처리
        print(f"Preprocessing 3DGS attributes for {scene_folder}...")
        features_3dgs = preprocess_3dgs_attributes(vertex_data_3dgs)

        # 3. 속성 분리 (use_features에 따라 동적으로 처리)
        feature_idx = 0
        if 'scale' in use_features:
            scale = features_3dgs[:, feature_idx:feature_idx+3]  # [N, 3]
            all_scales.append(scale)
            feature_idx += 3
        if 'opacity' in use_features:
            opacity = features_3dgs[:, feature_idx:feature_idx+1]  # [N, 1]
            all_opacities.append(opacity)
            feature_idx += 1
        if 'rotation' in use_features:
            rotation = features_3dgs[:, feature_idx:feature_idx+3]  # [N, 3]
            all_rotations.append(rotation)

    # 속성 병합
    min_max_dict = {}
    if 'scale' in use_features:
        all_scales = np.concatenate(all_scales, axis=0)
        min_max_dict['scale_min'] = np.min(all_scales, axis=0)
        min_max_dict['scale_max'] = np.max(all_scales, axis=0)
    if 'opacity' in use_features:
        all_opacities = np.concatenate(all_opacities, axis=0)
        min_max_dict['opacity_min'] = np.min(all_opacities, axis=0)
        min_max_dict['opacity_max'] = np.max(all_opacities, axis=0)
    if 'rotation' in use_features:
        all_rotations = np.concatenate(all_rotations, axis=0)
        min_max_dict['rotation_min'] = np.min(all_rotations, axis=0)
        min_max_dict['rotation_max'] = np.max(all_rotations, axis=0)

    # 결과 출력
    print("\nComputed Min/Max Values:")
    for key, value in min_max_dict.items():
        print(f"{key}: {value}")

    return min_max_dict

path_3dgs_root = "/home/knuvi/Desktop/song/data/3dgs_scans/3dgs_output"
# 최대/최소값 계산
min_max_dict = compute_3dgs_min_max(path_3dgs_root, use_features=('scale', 'opacity', 'rotation'))

# # 결과를 파일로 저장 (선택)
# import json
# with open(os.path.join(path_3dgs_root, "3dgs_min_max.json"), 'w') as f:
#     json.dump({k: v.tolist() for k, v in min_max_dict.items()}, f, indent=4)
# print(f"Min/Max values saved to {os.path.join(args.path_3dgs_root, '3dgs_min_max.json')}")


Processing scene0106_01...
All 3DGS normals are zero. Estimating normals...
After estimation - 3DGS normals norm: min=1.0000, max=1.0000, zero_count=0
Loaded 3DGS data from /home/knuvi/Desktop/song/data/3dgs_scans/3dgs_output/scene0106_01/point_cloud.ply: 320586 points
Preprocessing 3DGS attributes for scene0106_01...
Processing scene0297_00...
All 3DGS normals are zero. Estimating normals...
After estimation - 3DGS normals norm: min=1.0000, max=1.0000, zero_count=0
Loaded 3DGS data from /home/knuvi/Desktop/song/data/3dgs_scans/3dgs_output/scene0297_00/point_cloud.ply: 234785 points
Preprocessing 3DGS attributes for scene0297_00...
Processing scene0053_00...
All 3DGS normals are zero. Estimating normals...
After estimation - 3DGS normals norm: min=1.0000, max=1.0000, zero_count=0
Loaded 3DGS data from /home/knuvi/Desktop/song/data/3dgs_scans/3dgs_output/scene0053_00/point_cloud.ply: 114834 points
Preprocessing 3DGS attributes for scene0053_00...
Processing scene0484_01...
All 3DGS norm

In [ ]:
# 결과를 파일로 저장 (선택)
import json
with open(os.path.join('./', "3dgs_min_max.json"), 'w') as f:
    json.dump({k: v.tolist() for k, v in min_max_dict.items()}, f, indent=4)
print(f"Min/Max values saved to {os.path.join('./', '3dgs_min_max.json')}")